In [ ]:
import numpy as np
import pandas as pd
import random
from tensorflow import keras
import keras.backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.layers import BatchNormalization, Reshape, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import seaborn as sns
from tensorflow.keras.optimizers import Adam
import cv2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import os
import warnings

warnings.filterwarnings('ignore')

# Read the CSV file
labels_df = pd.read_csv('train.csv')
labels_df.head()

# Specify your dataset directory
train_dir = "E:\Internships\Codeclause\Project_1\Dataset\Train"
train_class = os.listdir(train_dir)

# Plot a few randomly selected images
rand_images = random.sample(os.listdir(train_dir), 15)

# Specify desired new size
width = 800
height = 600
dimension = (width, height)
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 10))

for i, im_name in enumerate(rand_images):
    im_path = os.path.join(train_dir, im_name)
    im = cv2.imread(im_path)

    if im is None:
        print(f"Failed to load image {im_name}")
        continue

    resized_image = cv2.resize(im, dimension, interpolation=cv2.INTER_AREA)
    resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

    plt.subplot(3, 5, i + 1)
    plt.imshow(resized_image)
    plt.axis('off')

plt.show()

# Split dataset into training and validation
validation_split = 0.8
idxs = np.random.permutation(range(len(labels_df))) < validation_split * len(labels_df)

train_labels = labels_df[idxs]
val_labels = labels_df[~idxs]
print(len(train_labels), len(val_labels))
train_labels

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1/255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

batch_size = 64

# Training data generator
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory=train_dir,
    x_col='ID',
    y_col='Class',
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(150, 150)
)

# Validation data generator
val_datagen = ImageDataGenerator(rescale=1/255)

val_generator = val_datagen.flow_from_dataframe(
    val_labels,
    directory=train_dir,
    x_col='ID',
    y_col='Class',
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(150, 150)
)

# MobileNetV2 model with custom output layer
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(150, 150, 3))
for layer in base_model.layers[:-10]:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

# Custom F1 score metric
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    f1_score,
]

# Callbacks for training
lrd = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.50, min_lr=1e-6)
mcp = ModelCheckpoint('model.h5')
es = EarlyStopping(verbose=1, patience=3)

# Model compilation
model.compile(optimizer='Adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=METRICS)

# Training
history = model.fit(train_generator, validation_data=val_generator, epochs=20, verbose=1, callbacks=[lrd, mcp, es])

# Plot training and validation metrics
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(history.history['val_loss']))

plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Save the model
model.save('my_model.h5')

# Convert the model to CoreML format
!pip install coremltools
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import coremltools as ct

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

# Load the trained model
keras_model = load_model('my_model.h5', custom_objects={'f1_score': f1_score})

# Convert the model to CoreML format
coreml_model = ct.convert(keras_model, inputs=[ct.ImageType()])

# Save the CoreML model
coreml_model.save('MyModel.mlpackage')
